Khi lần đầu tiên giới thiệu mạng thần kinh, chúng tôi tập trung vào các mô hình tuyến tính với một đầu ra duy nhất. Ở đây, toàn bộ mô hình chỉ bao gồm một nơ-ron duy nhất. Lưu ý rằng một nơ-ron đơn lẻ (i) nhận một số bộ đầu vào; (ii) tạo ra một đầu ra vô hướng tương ứng; và (iii) có một tập hợp các tham số liên quan có thể được cập nhật để tối ưu hóa một số chức năng mục tiêu quan tâm. Sau đó, khi chúng tôi bắt đầu nghĩ về các mạng có nhiều đầu ra, chúng tôi đã tận dụng phép toán số học được véc tơ hóa để mô tả toàn bộ một lớp tế bào thần kinh. Cũng giống như các nơ-ron riêng lẻ, các lớp (i) nhận một tập hợp các đầu vào, (ii) tạo ra các đầu ra tương ứng và (iii) được mô tả bằng một tập hợp các tham số có thể điều chỉnh. Khi chúng tôi làm việc thông qua hồi quy softmax, một lớp duy nhất chính là mô hình. Tuy nhiên, ngay cả khi chúng tôi giới thiệu MLP sau đó,

Thật thú vị, đối với MLP, cả mô hình và các lớp cấu thành của nó đều chia sẻ cấu trúc này. Toàn bộ mô hình nhận đầu vào thô (các tính năng), tạo đầu ra (dự đoán) và sở hữu các tham số (tham số kết hợp từ tất cả các lớp cấu thành). Tương tự như vậy, mỗi lớp riêng lẻ nhập đầu vào (được cung cấp bởi lớp trước) tạo đầu ra (đầu vào cho lớp tiếp theo) và sở hữu một tập hợp các tham số có thể điều chỉnh được cập nhật theo tín hiệu truyền ngược từ lớp tiếp theo.

Mặc dù bạn có thể nghĩ rằng nơ-ron, lớp và mô hình cung cấp cho chúng ta đủ sự trừu tượng để thực hiện công việc của mình, nhưng hóa ra chúng ta thường cảm thấy thuận tiện khi nói về các thành phần lớn hơn một lớp riêng lẻ nhưng nhỏ hơn toàn bộ mô hình. Ví dụ, kiến ​​trúc ResNet-152, cực kỳ phổ biến trong thị giác máy tính, sở hữu hàng trăm lớp. Các lớp này bao gồm các mẫu lặp lại của các nhóm lớp . Việc triển khai một lớp mạng như vậy tại một thời điểm có thể trở nên tẻ nhạt. Mối quan tâm này không chỉ là giả thuyết—các mẫu thiết kế như vậy là phổ biến trong thực tế. Kiến trúc ResNet được đề cập ở trên đã giành chiến thắng trong cuộc thi thị giác máy tính ImageNet và COCO năm 2015 cho cả nhận dạng và phát hiện ( He et al. , 2016 )và vẫn là một kiến ​​trúc phù hợp cho nhiều nhiệm vụ tầm nhìn. Các kiến ​​trúc tương tự trong đó các lớp được sắp xếp theo các mẫu lặp lại khác nhau hiện đang phổ biến trong các lĩnh vực khác, bao gồm xử lý ngôn ngữ tự nhiên và lời nói.

Để triển khai các mạng phức tạp này, chúng tôi giới thiệu khái niệm mô- đun mạng nơ-ron . Một mô-đun có thể mô tả một lớp duy nhất, một thành phần bao gồm nhiều lớp hoặc toàn bộ mô hình! Một lợi ích khi làm việc với tính trừu tượng của mô-đun là chúng có thể được kết hợp thành các tạo phẩm lớn hơn, thường là theo cách đệ quy. Điều này được minh họa trong Hình 6.1.1 . Bằng cách xác định mã để tạo các mô-đun có độ phức tạp tùy ý theo yêu cầu, chúng ta có thể viết mã nhỏ gọn đáng ngạc nhiên mà vẫn triển khai các mạng thần kinh phức tạp.

![Multiple layers are combined into modules, forming repeating patterns of larger models.](http://d2l.ai/_images/blocks.svg)

Từ quan điểm lập trình, một mô-đun được đại diện bởi một lớp . Bất kỳ lớp con nào của nó phải định nghĩa một phương pháp lan truyền thuận biến đầu vào của nó thành đầu ra và phải lưu trữ bất kỳ tham số cần thiết nào. Lưu ý rằng một số mô-đun hoàn toàn không yêu cầu bất kỳ tham số nào. Cuối cùng, một mô-đun phải sở hữu một phương pháp lan truyền ngược, cho mục đích tính toán độ dốc. May mắn thay, do một số bí quyết đằng sau hậu trường được cung cấp bởi sự phân biệt tự động (được giới thiệu trong Phần 2.5 ) khi xác định mô-đun của riêng chúng ta, chúng ta chỉ cần lo lắng về các tham số và phương pháp lan truyền thuận.


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

Để bắt đầu, chúng ta xem lại mã mà chúng ta đã sử dụng để triển khai MLP ( Mục 5.1 ). Đoạn mã sau tạo một mạng có một lớp ẩn được kết nối đầy đủ với 256 đơn vị và kích hoạt ReLU, tiếp theo là lớp đầu ra được kết nối đầy đủ với 10 đơn vị (không có chức năng kích hoạt).

In [13]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(16.7167)

Trong ví dụ này, chúng tôi đã xây dựng mô hình của mình bằng cách khởi tạo một nn.Sequential, với các lớp theo thứ tự mà chúng sẽ được thực thi dưới dạng đối số. Tóm lại, nn.Sequentialđịnh nghĩa một loại đặc biệt của Module, lớp trình bày một mô-đun trong PyTorch. Nó duy trì một danh sách có thứ tự các thành phần Modules. Lưu ý rằng mỗi lớp trong số hai lớp được kết nối đầy đủ là một thể hiện của lớp Linearmà bản thân nó là lớp con của Module. Phương thức lan truyền xuôi ( forward) cũng rất đơn giản: nó xâu chuỗi từng mô-đun trong danh sách lại với nhau, chuyển đầu ra của từng mô-đun làm đầu vào cho mô-đun tiếp theo. Lưu ý rằng cho đến bây giờ, chúng tôi đã gọi các mô hình của mình thông qua quá trình xây dựng net(X)để thu được kết quả đầu ra của chúng. Đây thực sự chỉ là cách viết tắt của net.`__call__(X)`.

# 6.1.1. A custom Module

Có lẽ cách dễ nhất để phát triển trực giác về cách hoạt động của một mô-đun là tự mình thực hiện mô-đun đó. Trước khi triển khai mô-đun tùy chỉnh của riêng mình, chúng tôi tóm tắt ngắn gọn chức năng cơ bản mà mỗi mô-đun phải cung cấp:

1. Nhập dữ liệu đầu vào làm đối số cho phương thức lan truyền về phía trước của nó.

2. Tạo đầu ra bằng cách yêu cầu phương thức lan truyền thuận trả về một giá trị. Lưu ý rằng đầu ra có thể có hình dạng khác với đầu vào. Ví dụ: lớp được kết nối đầy đủ đầu tiên trong mô hình của chúng tôi ở trên nhập đầu vào có kích thước tùy ý nhưng trả về đầu ra có kích thước 256.

3. Tính toán độ dốc của đầu ra đối với đầu vào của nó, có thể được truy cập thông qua phương thức lan truyền ngược của nó. Thông thường điều này xảy ra tự động.

4. Lưu trữ và cung cấp quyền truy cập vào các tham số cần thiết để thực hiện tính toán lan truyền về phía trước.

5. Khởi tạo các tham số mô hình khi cần thiết.

Trong đoạn mã sau, chúng tôi mã hóa một mô-đun từ đầu tương ứng với MLP với một lớp ẩn với 256 đơn vị ẩn và lớp đầu ra 10 chiều. Lưu ý rằng MLPlớp bên dưới kế thừa lớp đại diện cho một mô-đun. Chúng ta sẽ phụ thuộc rất nhiều vào các phương thức của lớp cha, chỉ cung cấp hàm tạo của riêng chúng ta (phương __init__ thức trong Python) và phương thức lan truyền xuôi.

In [3]:
class MLP(nn.Module):
    def __init__(self):

        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Trước tiên hãy tập trung vào phương pháp lan truyền thuận. Lưu ý rằng nó nhận Xlàm đầu vào, tính toán biểu diễn ẩn với chức năng kích hoạt được áp dụng và xuất các bản ghi của nó. Trong MLP triển khai này, cả hai lớp đều là các biến đối tượng. Để biết lý do tại sao điều này là hợp lý, hãy tưởng tượng khởi tạo hai MLP net1và net2, và đào tạo chúng trên các dữ liệu khác nhau. Đương nhiên, chúng tôi mong đợi chúng đại diện cho hai mô hình đã học khác nhau.

Chúng tôi khởi tạo các lớp của MLP trong hàm tạo và sau đó gọi các lớp này trên mỗi lần gọi phương thức lan truyền xuôi. Lưu ý một vài chi tiết chính. Đầu tiên, `__init__` phương thức tùy chỉnh của chúng tôi gọi phương thức của lớp cha mẹ `__init__` thông qua `super().__init__()`việc giúp chúng tôi tránh được việc khôi phục lại mã soạn sẵn áp dụng cho hầu hết các mô-đun. Sau đó, chúng tôi khởi tạo hai lớp được kết nối đầy đủ của mình, gán chúng cho self.hiddenvà self.out. Lưu ý rằng trừ khi chúng tôi triển khai một lớp mới, chúng tôi không cần phải lo lắng về phương thức lan truyền ngược hoặc khởi tạo tham số. Hệ thống sẽ tự động tạo các phương thức này. Hãy thử điều này ra.

In [4]:
net = MLP()
net(X).shape

torch.Size([2, 10])

Một đức tính quan trọng của sự trừu tượng hóa mô-đun là tính linh hoạt của nó. Chúng ta có thể phân lớp một mô-đun để tạo các lớp (chẳng hạn như lớp lớp được kết nối đầy đủ), toàn bộ mô hình (chẳng hạn như lớp MLP ở trên) hoặc các thành phần khác nhau có độ phức tạp trung gian. Chúng tôi khai thác tính linh hoạt này trong suốt các chương sau, chẳng hạn như khi giải quyết các mạng thần kinh tích chập.

# 6.1.2. Senquential Module

Bây giờ chúng ta có thể xem xét kỹ hơn cách Class Sequential hoạt động. Nhớ lại rằng nó Class Sequential được thiết kế để kết nối các mô-đun khác lại với nhau. Để xây dựng các phương thức đơn giản hóa của riêng mình MySequential, chúng ta chỉ cần xác định hai phương thức chính:

1. Một phương pháp để nối từng mô-đun một vào danh sách.

2. Một phương pháp lan truyền về phía trước để truyền đầu vào qua chuỗi các mô-đun, theo cùng thứ tự khi chúng được thêm vào.

Lớp sau MySequential cung cấp cùng chức năng của Sequential lớp mặc định.

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

Trong `__init__` phương thức, chúng tôi thêm mọi mô-đun bằng cách gọi add_modules phương thức. Các mô-đun này có thể được truy cập bằng childrenphương pháp sau này. Bằng cách này, hệ thống biết các mô-đun được thêm vào và nó sẽ khởi tạo đúng các tham số của từng mô-đun.

Khi MySequential phương thức lan truyền về phía trước của chúng ta được gọi, mỗi mô-đun được thêm vào sẽ được thực thi theo thứ tự mà chúng được thêm vào. Bây giờ chúng ta có thể triển khai lại MLP bằng cách sử dụng MySequentiallớp của chúng 

In [8]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

# 6.1.3. Mã thực thi trong Phương thức lan truyền chuyển tiếp

Lớp này Sequentiallàm cho việc xây dựng mô hình trở nên dễ dàng, cho phép chúng ta lắp ráp các kiến ​​trúc mới mà không cần phải định nghĩa lớp của riêng mình. Tuy nhiên, không phải tất cả các kiến ​​trúc đều là chuỗi cúc đơn giản. Khi cần tính linh hoạt cao hơn, chúng tôi sẽ muốn xác định các khối của riêng mình. Ví dụ: chúng ta có thể muốn thực thi luồng điều khiển của Python trong phương thức lan truyền chuyển tiếp. Hơn nữa, chúng ta có thể muốn thực hiện các phép toán tùy ý, không chỉ đơn giản dựa vào các lớp mạng thần kinh được xác định trước.

Bạn có thể nhận thấy rằng cho đến bây giờ, tất cả các hoạt động trong mạng của chúng tôi đều hoạt động dựa trên các kích hoạt và thông số của mạng. Tuy nhiên, đôi khi, chúng tôi có thể muốn kết hợp các thuật ngữ không phải là kết quả của các lớp trước đó cũng như các tham số có thể cập nhật. Chúng tôi gọi những tham số không đổi này . Ví dụ, nói rằng chúng ta muốn một lớp tính toán hàm $f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$, Với $\mathbf{x}$
là đầu vào, $\mathbf{w}$
là tham số của chúng tôi và $c$
là một số hằng số được chỉ định không được cập nhật trong quá trình tối ưu hóa. Vì vậy, chúng tôi thực hiện một Class FixedHiddenMLP như sau.

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

Trong FixedHiddenMLPmô hình này, chúng tôi triển khai một lớp ẩn có trọng số ( self.rand_weight) được khởi tạo ngẫu nhiên khi khởi tạo và sau đó không đổi. Trọng số này không phải là một tham số mô hình và do đó nó không bao giờ được cập nhật bằng lan truyền ngược. Sau đó, mạng chuyển đầu ra của lớp “cố định” này qua một lớp được kết nối đầy đủ.

Lưu ý rằng trước khi trả lại đầu ra, mô hình của chúng tôi đã làm điều gì đó bất thường. Chúng tôi đã chạy một vòng lặp while, thử nghiệm với điều kiện nó $\ell_1$ định mức lớn hơn $1$ và chia vectơ đầu ra của chúng tôi cho $2$
cho đến khi thỏa mãn điều kiện. Cuối cùng, chúng tôi đã trả về tổng của các mục trong X. Theo hiểu biết của chúng tôi, không có mạng thần kinh tiêu chuẩn nào thực hiện thao tác này. Lưu ý rằng thao tác cụ thể này có thể không hữu ích trong bất kỳ tác vụ nào trong thế giới thực. Mục đích của chúng tôi là chỉ cho bạn cách tích hợp mã tùy ý vào luồng tính toán mạng thần kinh của bạn.

In [11]:
net = FixedHiddenMLP()
net(X)

tensor(-0.3888, grad_fn=<SumBackward0>)

Chúng ta có thể trộn và kết hợp nhiều cách khác nhau để lắp ráp các mô-đun với nhau. Trong ví dụ sau, chúng tôi lồng các mô-đun theo một số cách sáng tạo.


In [12]:
"""
Kich thước đầu vào là: X(2,20), đầu ra lần lượt:
1. (2,64)
2.  ReLU
3. (2,32)
4.  ReLU
5. (2,16)
6. (2,20) 
7. (1) - Scalar
"""
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(-0.0902, grad_fn=<SumBackward0>)